In [2]:
!tree -d images

images
├── test
│   ├── JPEGImages
│   └── Visualization
├── train
│   ├── JPEGImages
│   └── Visualization
└── val
    ├── JPEGImages
    └── Visualization

9 directories


In [1]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [ ]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test
!python3 scripts/train_test_split.py -i images -o images -tr 0.1 -vr 0.15
!tree -d images

In [ ]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py images/train images/train/coco --labels training/labels.txt
!python3 scripts/labelme2coco.py images/test images/test/coco --labels training/labels.txt
!python3 scripts/labelme2coco.py images/val images/val/coco --labels training/labels.txt
!tree -d images

<img src="training/ejemplo_labels_txt.PNG">

In [ ]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes originales
!rm images/test/*
!mv images/test/coco/* images/test
!rm -rf images/test/coco 

!rm images/train/*
!mv images/train/coco/* images/train
!rm -rf images/train/coco 

!rm images/val/*
!mv images/val/coco/* images/val
!rm -rf images/val/coco 

!tree -d images

In [3]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir=images/train \
--val_image_dir=images/val \
--test_image_dir=images/test \
--train_annotations_file=images/train/annotations.json \
--val_annotations_file=images/val/annotations.json \
--testdev_annotations_file=images/test/annotations.json \
--output_dir=training/tfrecords \
--include_masks=True

I0902 08:29:11.932793 140572526618432 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0902 08:29:11.933215 140572526618432 create_coco_tf_record.py:412] 0 images are missing annotations.
I0902 08:29:11.933476 140572526618432 create_coco_tf_record.py:441] On image 0 of 117
I0902 08:29:35.454863 140572526618432 create_coco_tf_record.py:441] On image 100 of 117
I0902 08:29:39.719069 140572526618432 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I0902 08:29:39.724019 140572526618432 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0902 08:29:39.724163 140572526618432 create_coco_tf_record.py:412] 0 images are missing annotations.
I0902 08:29:39.724201 140572526618432 create_coco_tf_record.py:441] On image 0 of 24
I0902 08:29:45.580536 140572526618432 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I0902 08:29:45.583121 140572526618432 create_coco_tf_record.py:39

In [ ]:
# entrenamos el modelo usando estos tfrecords
!python3 object_detection/model_main.py --alsologtostderr \
--model_dir=training/ \
--pipeline_config_path=training/mask_rcnn_inception_v2_coco.config \
--num_train_steps=400000

W0902 08:54:14.865653 139718703343424 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 400000
I0902 08:54:14.865819 139718703343424 config_util.py:552] Maybe overwriting train_steps: 400000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0902 08:54:14.865867 139718703343424 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0902 08:54:14.865906 139718703343424 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0902 08:54:14.865944 139718703343424 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0902 08:54:14.865996 139718703343424 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_

Instructions for updating:
Use `tf.cast` instead.
W0902 08:54:26.486461 139718703343424 deprecation.py:323] From /home/bigdata/Documentos/proyecto_mario/tensorflow_object_detection_guide/object_detection/inputs.py:259: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
I0902 08:54:29.321695 139718703343424 estimator.py:1148] Calling model_fn.
Instructions for updating:
Please use `layer.__call__` method instead.
W0902 08:54:29.346276 139718703343424 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0902 08:54:30.275948 139718703343424 regularizers.py:99] 

INFO:tensorflow:Running local_init_op.
I0902 08:54:38.254704 139718703343424 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0902 08:54:38.453642 139718703343424 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into training/model.ckpt.
I0902 08:54:42.393664 139718703343424 basic_session_run_hooks.py:606] Saving checkpoints for 0 into training/model.ckpt.
2020-09-02 08:54:47.051249: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-09-02 08:54:50.459196: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-02 08:54:52.481956: W tensorflow/stream_executor/cuda/ptxas_utils.cc:116] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors